# BraTS 
In this Notebook we will demonstrate how to use the BraTS package to use top performing algorithms from the BraTS challenges.

---
## Getting Started

#### This tutorial requires:

   - Python 3.8+
   - Docker: Installation instructions on the official [website](https://docs.docker.com/get-docker/)


#### Optional but recommended:
   <!--CUDA 11.4+ (https://developer.nvidia.com/cuda-toolkit)-->
   - GPU with CUDA support (*otherwise CPU can be used for a some algorithms*) 
   - NVIDIA Container Toolkit: Refer to the [NVIDIA install guide](https://docs.nvidia.com/datacenter/cloud-native/container-toolkit/latest/install-guide.html) and the official [GitHub page](https://github.com/NVIDIA/nvidia-container-toolkit) 


In [ ]:
# Installations
!pip install brats matplotlib ipywidgets > /dev/null

%load_ext autoreload
%autoreload 2

If you installed the packages and requirements on your own machine, you can skip this section and start from the import section.

### Setup Colab environment (optional) 
Otherwise you can follow and execute the tutorial on your browser.
In order to start working on the notebook, click on the following button, this will open this page in the Colab environment and you will be able to execute the code on your own (*Google account required*).

<a target="_blank" href="https://colab.research.google.com/github/BrainLesion/tutorials/blob/main/BraTS/tutorial.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Now that you are visualizing the notebook in Colab, run the next cell to install the packages we will use. There are few things you should follow in order to properly set the notebook up:
1. Warning: This notebook was not authored by Google. Click on 'Run anyway'.
1. When the installation commands are done, there might be "Restart runtime" button at the end of the output. Please, click it.

If you run the next cell in a Google Colab environment, it will **clone the 'tutorials' repository** in your google drive. This will create a **new folder** called "tutorials" in **your Google Drive**.
All generated file will be created/uploaded to your Google Drive respectively.

After the first execution of the next cell, you might receive some warnings and notifications, please follow these instructions:
   - 'Permit this notebook to access your Google Drive files?' Click on 'Yes', and select your account.
   - Google Drive for desktop wants to access your Google Account. Click on 'Allow'.

Afterwards the "tutorials" folder has been created. You can navigate it through the lefthand panel in Colab. You might also have received an email that informs you about the access on your Google Drive.

In [ ]:
import sys

# Check if we are in google colab currently
try:
    import google.colab

    colabFlag = True
except ImportError as r:
    colabFlag = False

# Execute certain steps only if we are in a colab environment
if colabFlag:
    # Create a folder in your Google Drive
    from google.colab import drive

    drive.mount("/content/drive")
    # clone repository and set path
    !git clone https://github.com/BrainLesion/tutorials.git /content/drive/MyDrive/tutorials
    BASE_PATH = "/content/drive/MyDrive/tutorials/BraTS/"
    sys.path.insert(0, BASE_PATH)

else:  # normal jupyter notebook environment
    BASE_PATH = "./"  # current working directory would be BraTs-Toolkit anyways if you are not in colab

## Imports

In [ ]:
from pathlib import Path
from brats import AdultGliomaSegmenter
from brats.constants import AdultGliomaAlgorithms
import utils  # local file

---

## Data

AURORA expects *preprocessed* input data as NIfTI file or NumPy Array (*preprocessed* meaning the files should be co-registerend, skullstripped and in SRI-24 space).

In this example we provide sample data from the [ASNR-MICCAI BraTS Brain Metastasis Challenge](https://www.synapse.org/#!Synapse:syn51156910/wiki/622553), which is already preprocessed in the `AURORA/data` folder in the form of 4 modalities of the same brain (T1, T1C, T2, FLAIR). To get an intuition of the data, one example slice of the 3D scans is visualized below.

For your own data:
If the data is *not* preprocessed yet, consider using our [BrainLes preprocessing](https://github.com/BrainLesion/preprocessing) package (or its predecessor [BraTS-Toolkit](https://github.com/neuronflow/BraTS-Toolkit)).


In [ ]:
subject = "BraTS-GLI-00001-000"
data_path = Path(BASE_PATH) / "data"
subject_path = data_path / subject
utils.visualize_data(data_path, subject_id=subject)

## Using BraTS

### Minimal example using default settings

In [ ]:
segmenter = AdultGliomaSegmenter()
segmenter.infer_single(
    t1c=subject_path / f"{subject}-t1c.nii.gz",
    t1n=subject_path / f"{subject}-t1n.nii.gz",
    t2f=subject_path / f"{subject}-t2f.nii.gz",
    t2w=subject_path / f"{subject}-t2w.nii.gz",
    output_file="segmentation.nii.gz",
)

### Visualize results

The segementation comprise of the
TODO


In [ ]:
utils.visualize_segmentation(
    modality_file=subject_path / f"{subject}-t1c.nii.gz",
    segmentation_file="segmentation.nii.gz"
)

## Batch processing

BraTS allows to run an algorithm for a single set of input images (t1n, t1c, t2f, t2w of the same patient) or for multiple subjects.
Each of the available classes provides methods for both: 
- `.infer_single(...)` that takes in the paths to the required input modalities and a path to store the result
- `.infer_batch(...)` that takes in a path to a data folder containing multiple sets of subjects and a path to an output folder to store the results

#### The example below shows how to perform a batch inference
The sets of subject inputs need to be stored in a specific structure to be recognized by the package:
```
data_folder
┣ A
┃ ┣ A-t1c.nii.gz
┃ ┣ A-t1n.nii.gz
┃ ┣ A-t2f.nii.gz
┃ ┗ A-t2w.nii.gz
┣ B
┃ ┣ B-t1c.nii.gz
┃ ┣ ...
```

In [ ]:
output_path = Path("outputs")

segmenter = AdultGliomaSegmenter(cuda_devices="4")
segmenter.infer_batch(
    data_folder=data_path,
    output_folder=output_path,
)

print([path.name for path in output_path.iterdir()])

## Advanced Usage

By default the algorithm that won the most recent challenge will be run on the first available GPU. This behavior and other options can be adapted, e.g.:
- Select a different algorithm from the available constants (Enum classes for each challenge) with the `algorithm` parameter
- Select a specific GPU if multiple are available with the `cuda_decives` parameter
- Force CPU execution with the `force_cpu`flag (will cause an exception for many algorithms since many do not support CPU execution)
- Save the generated logs in a log file with the `log_file` parameter

In [ ]:
segmenter = AdultGliomaSegmenter(
    algorithm=AdultGliomaAlgorithms.BraTS23_3, # Use the 3rd placed algorithm of the Adult Glioma BraTS 2023 challenge
    cuda_devices="4", # Select GPU device with ID 4
    force_cpu=False, # default, could be set to True to force CPU
)

segmenter.infer_single(
    t1c=subject_path / f"{subject}-t1c.nii.gz",
    t1n=subject_path / f"{subject}-t1n.nii.gz",
    t2f=subject_path / f"{subject}-t2f.nii.gz",
    t2w=subject_path / f"{subject}-t2w.nii.gz",
    output_file="segmentation.nii.gz",
    log_file="segmentation.log", # Save the logs in a new filed called `segmentation.log`
)

## Algorithms from other Challenges

BraTS provides the algorithms from all available recent BraTS Challenges, i.e.:
- Adult Glioma Segmentation
- BraTS-Africa Segmentation
- Meningioma Segmentation
- Brain Metastases Segmentation
- Pediatric Tumors Segmentation
- Inpainting

The package provides a separate class and algorithm constants for each of the challenges.<br>
The examples above were demonstrated using the class and constants of the Adult Glioma Segmentation challenge.

In an identical way you can use:
- `MeningiomaSegmenter` class with `MeningiomaAlgorithms`
- `PediatricSegmenter` class with `PediatricAlgorithms`
- etc.

In [ ]:
# e.g. for the Meningioma Algorithms
from brats import MeningiomaSegmenter
from brats.constants import MeningiomaAlgorithms

segmenter = MeningiomaSegmenter(
    algorithm=MeningiomaAlgorithms.BraTS23_2, cuda_devices="4"
)
segmenter.infer_batch(
    data_folder=data_path, output_folder=output_path, log_file="test.log"
)